In [1]:
import pandas as pd
import uproot as ur
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
file = ur.open('/home/jmsardain/JetCalib/Akt4EMTopo.topo_cluster.root')["ClusterTree"]
df = file.arrays(library="pd")
df1 = df.head(1000000)

In [8]:
clusterE = [] 
clusterEta = []
cluster_time = []
labels = [] 

In [4]:
## Get all event numbers, but just once per event, so drop duplicates
eventNumbers = np.array(df1.eventNumber.drop_duplicates())

In [6]:
eventNumbers = eventNumbers[0:10]
eventNumbers

array([47081445, 47081807, 47081814, 47081454, 47081148, 47081685,
       47081237, 47081882, 47081278, 47081045])

In [9]:
for event in eventNumbers:
    a = list(df[df["eventNumber"]==event].clusterE.values)
    b = list(df[df["eventNumber"]==event].clusterEtaCalib.values)
    c = list(df[df["eventNumber"]==event].cluster_time.values)
    d = list(np.where(np.array(df[df["eventNumber"]==event].cluster_ENG_CALIB_TOT.values) == 0, 1, 0))
    clusterE.append(a)
    clusterEta.append(b)
    cluster_time.append(c)
    labels.append(d)
    pass

In [23]:
import torch
from torch_geometric.data import Data

graph_list = []
for i in range(len(clusterE)):
    num_nodes = len(clusterE[i])
    edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(i+1, num_nodes)], dtype=torch.long).t().contiguous()
    vec = []
    vec.append(np.array([clusterE[i], clusterEta[i], cluster_time[i]]).T)
    x = torch.tensor(vec, dtype=torch.float)
    
    graph = Data(x=x, edge_index=edge_index, y=labels[i])
    graph_list.append(graph)

# Printing the graph list
for idx, graph in enumerate(graph_list):
    if idx>3: continue
    print(f"Graph {idx+1}:")
    print(graph)
    print()

Graph 1:
Data(x=[1, 39, 3], edge_index=[2, 741], y=[39])

Graph 2:
Data(x=[1, 12, 3], edge_index=[2, 66], y=[12])

Graph 3:
Data(x=[1, 20, 3], edge_index=[2, 190], y=[20])

Graph 4:
Data(x=[1, 17, 3], edge_index=[2, 136], y=[17])



In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
class GCNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# class GCNModel(nn.Module):
#     def __init__(self, in_channels, hidden_channels, num_classes):
#         super(GCNModel, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, num_classes)

#     def forward(self, x_batch, edge_index_batch):
#         x_batch = self.conv1(x_batch, edge_index_batch)
#         x_batch = torch.relu(x_batch)
#         x_batch = self.conv2(x_batch, edge_index_batch)
#         return x_batch
        
# Initialize model, optimizer, and loss function
model = GCNModel(in_channels=3, hidden_channels=64, num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion.to(device)

CrossEntropyLoss()

In [55]:
from torch.utils.data import DataLoader
dataloader = DataLoader(graph_list, batch_size=1028, shuffle=True)

In [56]:

# Training loop
for epoch in range(100):
    for data in graph_list:
        data = data.to(device)
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        out = out.view(-1, out.shape[-1])
        labels = torch.tensor(data.y, dtype=torch.long).to(device) 
        # print(out.shape)
        # print(labels.shape)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()


In [57]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    
    for data in graph_list:
        data = data.to(device)  # Move batch data to the same device as model
        
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        out = out.view(-1, out.shape[-1])
        labels = torch.tensor(data.y, dtype=torch.long).to(device) 
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')


Epoch [1/100], Loss: 2.4406
Epoch [2/100], Loss: 2.4170
Epoch [3/100], Loss: 2.4040
Epoch [4/100], Loss: 2.4099
Epoch [5/100], Loss: 2.3975
Epoch [6/100], Loss: 2.4531
Epoch [7/100], Loss: 2.4391
Epoch [8/100], Loss: 2.4701
Epoch [9/100], Loss: 2.5697
Epoch [10/100], Loss: 2.5717
Epoch [11/100], Loss: 2.5406
Epoch [12/100], Loss: 2.7763
Epoch [13/100], Loss: 2.6670
Epoch [14/100], Loss: 2.4771
Epoch [15/100], Loss: 2.4939
Epoch [16/100], Loss: 2.4619
Epoch [17/100], Loss: 2.4110
Epoch [18/100], Loss: 2.3991
Epoch [19/100], Loss: 2.4076
Epoch [20/100], Loss: 2.3877
Epoch [21/100], Loss: 2.3821
Epoch [22/100], Loss: 2.3854
Epoch [23/100], Loss: 2.3785
Epoch [24/100], Loss: 2.3764
Epoch [25/100], Loss: 2.3722
Epoch [26/100], Loss: 2.3767
Epoch [27/100], Loss: 2.3729
Epoch [28/100], Loss: 2.3767
Epoch [29/100], Loss: 2.3701
Epoch [30/100], Loss: 2.3900
Epoch [31/100], Loss: 2.3826
Epoch [32/100], Loss: 2.4054
Epoch [33/100], Loss: 2.4071
Epoch [34/100], Loss: 2.5090
Epoch [35/100], Loss: 2

In [62]:
# Evaluate the trained model on the test graph
model.eval()
with torch.no_grad():
    test_out = model(graph_list[1].x, graph_list[1].edge_index)
    predicted_labels = test_out.argmax(dim=1)

print("Predicted labels for the test graph:", predicted_labels.tolist())

Predicted labels for the test graph: [[2, 0]]


In [63]:
len(list(np.where(np.array(df[df["eventNumber"]==47081807].cluster_ENG_CALIB_TOT.values) == 0, 1, 0)))

12

In [59]:
eventNumbers

array([47081445, 47081807, 47081814, 47081454, 47081148, 47081685,
       47081237, 47081882, 47081278, 47081045])